In [93]:
import pandas as pd
import numpy as np
import re


# load dataset
df_country = pd.read_excel("GSAF5.xls")

df_country ["id"] = range(0,len(df_country))
df_country

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,id
0,2024-09-16 00:00:00,2024.0,Unprovoked,Morocco,Southern Morocco,West of Dakhla,Swimming - jumped off yacht,German Tourist,F,30,...,Andy Currie: Moroccan World News: Sky News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2024-08-26 00:00:00,2024.0,Unprovoked,Jamaica,Montego Bay,Falmouth,Spearfishing,Jahmari Reid,M,16,...,Todd Smith: Daily Mail UK: Sky News: People .com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2024-08-06 00:00:00,2024.0,Unprovoked,Belize,Gulf of Honduras,Central America,Swimming - Diving,Annabelle Carlson,F,15,...,Kevin McMurray Tracking Sharks.com New York Po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2024-07-23 00:00:00,2024.0,Unprovoked,Australia,NSW,"North Shore Beach, Port Macquarie",Surfing,Kai McKenzie,M,23,...,Nine.com.au: ABC.Net.Au Johannes Marchand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2024-07-18 00:00:00,2024.0,Unprovoked,Australia,Western Australia,Trigg beach Sterling,Surfing,Ryan Lowther,M,41,...,Daily Mail: Sky News: The West Australian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6965,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,...,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN,6965
6966,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,...,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN,6966
6967,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,...,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN,6967
6968,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,...,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN,6968


In [94]:
df_country['Country_Comment']= 'Check'
df_country['Country']= df_country['Country'].str.strip()
countryregex = r'^[A-Z\s]+$'
countryregex2 = r'^(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)?$|^(?:[A-Z][a-z]*\s*)+$'
countryregex3 = r'^[A-Z\s]+ \/ [A-Z\s]+$'
countryregex4 = r'^[A-Z]+(?:\s[A-Z]+)*$'
countryregex5 = r'^[A-Z\s&]+$'

df_country.loc[
    df_country['Country'].str.match(countryregex)==True,
    'Country_Comment'
    ]= "Country_ok"

pattern2 = r'^(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)?$|^(?:[A-Z][a-z]*\s*)+$'

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex2)==True),
    'Country_Comment'
    ] = 'Countries haveing First/Second word Capital Letter and rest smalls letters'


df_country.loc[
    (df_country ['Country_Comment']=='Check') &
    (df_country ['Country'].str.match(countryregex3)==True),
    'Country_Comment'
    ] = 'Countries split by / but with capital letters'

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex4)==True),
    'Country_Comment'
    ] = "Country in capitals containing ?"

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex5)==True),
    'Country_Comment'
    ] = "Country in capitals separated by &"

pd.pivot_table(df_country, index = 'Country_Comment',values = 'Country',aggfunc='count')

df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Country in capitals separated by &')),
    ["Country"]
              ] =  df_country.loc[
                  ((df_country['Country_Comment'] != 'Country_ok') & 
                   (df_country['Country_Comment'] == 'Country in capitals separated by &')),
                   ['Country']
                   ].replace("&", "AND", regex=True)

df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters')),
    ["Country","Location","State"]
             ]
##CLEANING SPECIAL CHARACTERS

df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['State']=="South Sinai, Gulf of Aqaba")),
    ["Country"]] = "EGYPT"
df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['State']=="Enroute from Suez to Aden (Yemen)")),
    ["Country"]] = "RED SEA"
df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['State']== "Shatt-al-Arab River")),
    ["Country"]] = "IRAQ"
df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['Country']== "SOLOMON ISLANDS / VANUATU")),
    ["Country"]] = "SOLOMON ISLANDS"

df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['Location']== "Kralievica")),
    ["Country"]] = "CROATIA"

df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries split by / but with capital letters') &
    (df_country['State']=="Fernando Po Island")),
    ["Country"]] = "EQUATORIAL GUINEA"

#CLENAING SMALL LETTERS TO CAPITALS
df_country.loc[
    ((df_country['Country_Comment']!='Country_ok') & 
    (df_country['Country_Comment']=='Countries haveing First/Second word Capital Letter and rest smalls letters')),
    "Country"
    ] =  df_country.loc[
        ((df_country['Country_Comment']!='Country_ok') & 
         (df_country['Country_Comment']=='Countries haveing First/Second word Capital Letter and rest smalls letters')),
         "Country"
         ].str.upper() 

### 2nd Regex filternig 
df_country['Country_Comment']= 'Check'
df_country['Country']= df_country['Country'].str.strip()
countryregex = r'^[A-Z\s]+$'
countryregex2 = r'^(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)?$|^(?:[A-Z][a-z]*\s*)+$'
countryregex3 = r'^[A-Z\s]+ \/ [A-Z\s]+$'
countryregex4 = r'^[A-Z]+(?:\s[A-Z]+)*$'
countryregex5 = r'^[A-Z\s&]+$'

df_country.loc[
    df_country['Country'].str.match(countryregex)==True,
    'Country_Comment'
    ]= "Country_ok"

pattern2 = r'^(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)?$|^(?:[A-Z][a-z]*\s*)+$'

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex2)==True),
    'Country_Comment'
    ] = 'Countries haveing First/Second word Capital Letter and rest smalls letters'


df_country.loc[
    (df_country ['Country_Comment']=='Check') &
    (df_country ['Country'].str.match(countryregex3)==True),
    'Country_Comment'
    ] = 'Countries split by / but with capital letters'

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex4)==True),
    'Country_Comment'
    ] = "Country in capitals containing ?"

df_country.loc[
    (df_country['Country_Comment']=='Check') &
    (df_country['Country'].str.match(countryregex5)==True),
    'Country_Comment'
    ] = "Country in capitals separated by &"


#filtering all countries that are OK >>> **Testing** DataFrame *** NaN(s) needs to be fixed
df_country = df_country.loc[
    (df_country['Country_Comment'] =='Country_ok') 
     ] 

In [95]:
df_country["Country"] = df_country["Country"].str.title()
df_country.head()

C:\Users\xavie\AppData\Local\Temp\ipykernel_4428\737727424.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22,id,Country_Comment
0,2024-09-16 00:00:00,2024.0,Unprovoked,Morocco,Southern Morocco,West of Dakhla,Swimming - jumped off yacht,German Tourist,F,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Country_ok
1,2024-08-26 00:00:00,2024.0,Unprovoked,Jamaica,Montego Bay,Falmouth,Spearfishing,Jahmari Reid,M,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Country_ok
2,2024-08-06 00:00:00,2024.0,Unprovoked,Belize,Gulf of Honduras,Central America,Swimming - Diving,Annabelle Carlson,F,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Country_ok
3,2024-07-23 00:00:00,2024.0,Unprovoked,Australia,NSW,"North Shore Beach, Port Macquarie",Surfing,Kai McKenzie,M,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Country_ok
4,2024-07-18 00:00:00,2024.0,Unprovoked,Australia,Western Australia,Trigg beach Sterling,Surfing,Ryan Lowther,M,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Country_ok


In [96]:
df_sharks.to_csv(r"C:\Users\xavie\OneDrive\Documentos\Ironhack\Week 2\Quest\clean_country.csv", index=False)

NameError: name 'df_sharks' is not defined

# Coordinates API

In [19]:
# Create a new df with only the countries, unique values

country_counts_coo = df_country['Country'].value_counts().reset_index()
country_counts_coo.columns = ['Country', 'Count']

country_counts_coo

,Country,Count
0,USA,2550
1,AUSTRALIA,1485
2,SOUTH AFRICA,598
3,NEW ZEALAND,145
4,BAHAMAS,137
...,...,...
185,ADMIRALTY ISLANDS,1
186,FEDERATED STATES OF MICRONESIA,1
187,BRITISH ISLES,1
188,WESTERN SAMOA,1


In [21]:
# importing geopy library and Nominatim class
from geopy.geocoders import Nominatim

def capital_coordinates_lat_api(country):
    try:
        coordinates = Nominatim(user_agent="Geopy Library").geocode(country)
        return coordinates.latitude
    except:
        return None

def capital_coordinates_lng_api(country):
    try:
        coordinates = Nominatim(user_agent="Geopy Library").geocode(country)
        return coordinates.longitude
    except:
        return None



country_counts_coo["coo_lat"] = country_counts_coo["Country"].apply(capital_coordinates_lat_api)
country_counts_coo["coo_lng"] = country_counts_coo["Country"].apply(capital_coordinates_lng_api)



In [63]:
country_counts_coo

,Country,Count,coo_lat,coo_lng,Color
0,USA,2550,39.783730,-100.445882,red
1,AUSTRALIA,1485,-24.776109,134.755000,red
2,SOUTH AFRICA,598,-28.816624,24.991639,red
3,NEW ZEALAND,145,-41.500083,172.834408,red
4,BAHAMAS,137,24.773655,-78.000055,red
...,...,...,...,...,...
185,ADMIRALTY ISLANDS,1,-2.103264,146.932545,blue
186,FEDERATED STATES OF MICRONESIA,1,8.606235,151.832744,blue
187,BRITISH ISLES,1,54.060000,-4.800000,blue
188,WESTERN SAMOA,1,-13.769390,-172.120050,blue


In [89]:
# Create a world map with the incidents by coordinates, size by count of instances
import plotly.express as px

# Determine the top countries
top_countries = country_counts_coo.nlargest(5, 'Count')['Country'].tolist()

# Create a new column for colors
country_counts_coo['Color'] = country_counts_coo['Country'].apply(
    lambda x: 'Top 5' if x in top_countries else "Other"  # Change 'blue' to any color you prefer for others
)

fig = px.scatter_geo(country_counts_coo,
                    lat=country_counts_coo.coo_lat,
                    lon=country_counts_coo.coo_lng,
                    size = "Count",
                    size_max=20,
                    hover_name="Country",
                    color="Color",
                    color_discrete_map={'Top 5': '#ed472b', 'Other': '#636efa'},
                    width=1000,
                    height=500)

fig.update_traces(marker=dict(sizemin=4))


fig.show()